In [9]:
import pandas as pd
import numpy as np
import math
import os
from PyRadioLoc.Utils.GeoUtils import GeoUtils
%run utils.ipynb

In [15]:
FILES = [
    './result/teorico_med.csv',
    './result/teorico_fingerprint_30.csv',
    './result/teorico_fingerprint_40.csv',
    './result/teorico_med_fingerprint_30.csv',
    './result/teorico_med_fingerprint_40.csv'
]

# 1. Análise dos dados

In [59]:
data = pd.read_csv("./database/LocTreino_Equipe_4.csv")
data.describe()

,lat,lon,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,pontoId
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,-8.052194,-34.952254,-88.894304,-92.674192,-89.101067,-88.582943,-90.587305,-89.274726,-87.837028,-91.090342,-93.173225,4.617333,3.962000,3.446667,8134.680000
std,0.003181,0.004043,6.244573,5.709887,6.129258,12.700474,7.978007,6.903897,7.899230,9.982229,8.029798,1.467084,1.648343,1.756857,889.653236
min,-8.059550,-34.959850,-110.210000,-105.000000,-104.485000,-107.010000,-104.470000,-104.540000,-113.300000,-115.630000,-105.130000,1.000000,1.000000,0.000000,6616.000000
25%,-8.055133,-34.955153,-92.876190,-97.102857,-93.053661,-97.883333,-96.736250,-94.300000,-92.230000,-98.098000,-97.255000,3.000000,3.000000,2.000000,7360.750000
50%,-8.051960,-34.952720,-89.488571,-93.606429,-90.181429,-93.146667,-91.935000,-90.656905,-89.370000,-94.332000,-96.281250,5.000000,4.000000,3.000000,8158.500000
75%,-8.049527,-34.948518,-85.567500,-89.400000,-86.273929,-83.137500,-86.860521,-85.638929,-84.170000,-85.380000,-93.387500,6.000000,5.000000,5.000000,8924.250000
max,-8.046420,-34.945110,-61.380000,-65.390000,-62.320000,-36.960000,-55.550000,-60.480000,-52.690000,-55.870000,-50.100000,8.000000,7.000000,7.000000,9679.000000


# 2. Histograma dos erros (em metros)

# 3. Boxplot dos erros (em metros)

# 4. Mapa de comparação - posições preditas vs. posições reais

# 5. Erros de localização médio, mínimo, máximo e desvio padrão

## 5.1 Erro modelo teórico e com medições

In [16]:
def calculate_errors(df_real, df_predict):
    """
    Retorna uma lista contendo os erros em metros de todos os pontos no real e na predição
    """
    df_merged = merge_on_pontoId(df_predict, df_real)
    errors = []
    
    for idx, point in df_merged.iterrows():
        distance = GeoUtils.distanceInKm(point.lat_x, point.lon_x, point.lat_y, point.lon_y)
        errors.append(round(distance * 1000, 2))
        
    return errors

def calculate_theorical_errors():
    df_real = pd.read_csv("./database/LocTreino_Equipe_4.csv")
    dict_errors = {}
    
    for file in FILES:
        filename = file.split("/")
        filename = "/".join(filename[2:]).replace(".csv", "")
        df_predict = pd.read_csv(file)
        dict_errors[filename] = calculate_errors(df_real, df_predict)
    
    return pd.DataFrame(dict_errors)

In [17]:
# Gera o dataframe contendo os erros
errors_df = calculate_theorical_errors()
round(errors_df.describe(), 2)

,teorico_med,teorico_fingerprint_30,teorico_fingerprint_40,teorico_med_fingerprint_30,teorico_med_fingerprint_40
count,150.00,150.00,150.00,150.00,150.00
mean,58.26,833.12,828.49,58.26,58.26
std,86.41,309.19,310.21,86.41,86.41
min,1.56,78.55,67.86,1.56,1.56
25%,7.88,639.76,639.02,7.88,7.88
50%,24.38,789.35,785.88,24.38,24.38
75%,54.05,1021.84,1020.77,54.05,54.05
max,458.76,1527.27,1526.12,458.76,458.76
